# Table for comparison with regime and without regime

Instruction: this programme is to compare the result with and without regime switching based on the observed sample frequency. The result is as shown on the bottem of the page.

In [151]:
def Phi1(T,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2): 
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    #Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82)
    #Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82)
    #Regime = Regime2(Q);
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    #Jump1 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82,0.05,0.086,0.)
    #Jump2 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82,0.05,0.086,0.3)
    #####################################################################################
    # ###############################Numerical Integration########################
    n = 10 # time step of integration
    X = np.linspace(T-Delta,T,n+1)
    phi1_1_2j = [];phi1_1_1j = [];phi1_1_0j = [];
    phi1_2_2j = [];phi1_2_1j = [];phi1_2_0j = []; 
    
    for i in range(len(X)):
        x1 = Jump1.L(-2j,X[i]); x2=Jump1.L(-1j,X[i]);x3=Jump1.L(0,X[i]);
        phi1_1_2j.append(x1); phi1_1_1j.append(x2); phi1_1_0j.append(x3);
        y1 = Jump2.L(-2j,X[i]); y2=Jump2.L(-1j,X[i]);y3=Jump2.L(0,X[i]);
        phi1_2_2j.append(y1); phi1_2_1j.append(y2);phi1_2_0j.append(y3); 
        
        
    phI1_1_2j = np.trapz(phi1_1_2j,dx=Delta/n);phI1_2_2j = np.trapz(phi1_2_2j,dx=Delta/n);
    phI1_1_1j = np.trapz(phi1_1_1j,dx=Delta/n);phI1_2_1j = np.trapz(phi1_2_1j,dx=Delta/n);
    phI1_1_0j = np.trapz(phi1_1_0j,dx=Delta/n);phI1_2_0j = np.trapz(phi1_2_0j,dx=Delta/n);
  
    #################################Diagonal Matrix#########################################
    phi1_Matrix_2j = np.diag(np.array([phI1_1_2j,phI1_2_2j]));
    phi1_Matrix_1j = np.diag(np.array([phI1_1_1j,phI1_2_1j]));
    phi1_Matrix_0j = np.diag(np.array([phI1_1_0j,phI1_2_0j]));
    
    #######################Phi1_characteristic function#####################################
    Phi1_2j = Regime.character(phi1_Matrix_2j,T-Delta,T);
    Phi1_1j = Regime.character(phi1_Matrix_1j,T-Delta,T);
    Phi1_0j = Regime.character(phi1_Matrix_0j,T-Delta,T);
    return Phi1_2j, Phi1_1j, Phi1_0j

In [152]:
def Phi2(T,AF,Q,Delta,Stock1,Stock2,Regime):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    #
    #Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82)# S0, y0, AF, r, sigma, a, b, N, rho
    #Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82)
    #Regime = Regime2(Q);
    #
    ###############################################################################
    
    n = 10 # time step of integration
    X = np.linspace(0,T-Delta,n+1)
    phi2_1_2j = [];phi2_2_2j = [];
    
    for i in range(len(X)):
        H1 = Stock1.H(X[i]);H2 = Stock2.H(X[i]);
        x = Stock1.a*Stock1.b*H1;y = Stock2.a*Stock2.b*H2;
        phi2_1_2j.append(x);phi2_2_2j.append(y);
        #print(H1,X[i],T-Delta)
        
    phI2_1_2j = np.trapz(phi2_1_2j,dx=(T-Delta)/n);phI2_2_2j = np.trapz(phi2_2_2j,dx=(T-Delta)/n);
    phi2_Matrix = np.diag(np.array([phI2_1_2j,phI2_2_2j]))
    Phi2 = Regime.character(phi2_Matrix,0,T-Delta)
    return Phi2,Stock1.H(0)

In [153]:
def regime_VS(AF):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
   
    Delta = 1/AF
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])#transition matrix
    #Stock1 = Stock(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    #Stock2 = Stock(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82)
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    #Jump1 = Jump_Merton(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82,0.05,0.086,0.)
    #Jump2 = Jump_Merton(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82,0.06,0.086,0.3)
    Regime = Regime2(Q);
    ################################################################################
    
    U = np.array([0,0])#initialize
    T = 1
    for k in range(0,AF*T):
        t_k = (k+1)*Delta
        Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
        Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82)
        Jump1 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82,0.05,0.086,0.3)
        Jump2 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82,0.05,0.086,0.)
        R = np.diag([np.exp(Stock1.r*Delta),np.exp(Stock2.r*Delta)])# matrix of interest rate
        Phi1_2j,Phi1_1j,Phi1_0j = Phi1(t_k,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2) 
        Phi2_,H1 = Phi2(t_k,AF,Q,Delta,Stock1,Stock2,Regime)
        if t_k == Delta:
            M = Stock1.M(-2j,0)
            uk = Phi1_2j[1]*np.exp(M*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
        else:
            uk = np.multiply(Phi1_2j[1],Phi2_[1])*np.exp(H1*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
            
        U = U+Uk
        
        
        
       
    K = (U/T)*10000
    return K

In [154]:
def discreet(AF):
    
    
    Kvar = []
    
    for t in AF:
        K = regime_VS(t)
        Kvar.append(K)
     
    return(Kvar)


In [165]:

def Continuous():
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    
    Stock1 = Stock(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    Stock2 = Stock(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82)
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    Jump1 = Jump_Merton(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82,0.05,0.086,0.2)
    Jump2 = Jump_Merton(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82,0.05,0.086,0.1)
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])
    Regime = Regime2(Q);T=1;


    n = 10 # time step of integration
    X = np.linspace(0,1,n+1)
    Phi_Matrix=[]
    E = np.ones(Q.ndim)                     
    for i in range(len(X)):
        v1 = Stock1.mean(X[i]);v2 = Stock2.mean(X[i])
        phi = np.diag(np.array([v1,v2]))
        pphi = np.matmul(linalg.expm(Q.transpose()*(X[i])),phi)
        Phi = np.matmul(E,pphi)
        Phi_Matrix.append(Phi)
       # print(Phi)
        
    
    a = list(zip(*Phi_Matrix))
   # print(a)
    V1 = np.trapz(a,dx=0.1);
   # print(V1)
    j1 = Jump1.mean_com(); j2 = Jump2.mean_com();
    V2 = np.diag(np.array([j1,j2]));
    J =  np.matmul(linalg.expm(Q*T),V2);
    print(J)
    Kvar = ((1-np.exp(-Stock1.a))/Stock1.a)*Stock1.y0+Stock1.a*V1+np.matmul(J,E);
    print(j2+np.matmul(E,J),Kvar)
    #print(Stock1.a*V1)


    return(Kvar*10000)

In [156]:
def Continuous2():
   ####Continuous solution without regime switching
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    Stock1 = Stock(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    Stock2 = Stock1
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    Jump1 = Jump_Merton(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82,0.05,0.086,0.1)
    Jump2 = Jump1
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])
    Regime = Regime2(Q);T=1;


    n = 101 # time step of integration
    X = np.linspace(0,1,n+1)
    Phi_Matrix=[]
    E = np.ones(Q.ndim)                     
    for i in range(len(X)):
        v1 = Stock1.mean(X[i]);v2 = Stock2.mean(X[i])
        phi = np.diag(np.array([v1,v2]))
        pphi = np.matmul(linalg.expm(Q.transpose()*(X[i])),phi)
        Phi = np.matmul(E,pphi)
        Phi_Matrix.append(Phi)
        #print(Phi)
        
    
    a = list(zip(*Phi_Matrix))
    #print(a)
    V1 = np.trapz(a,dx=0.01);
    j1 = Jump1.mean_com(); j2 = Jump2.mean_com();
    V2 = np.diag(np.array([j1,j2]));
    J = np.matmul(linalg.expm(Q.transpose()*T),V2);
    #Kvar = ((1-np.exp(-Stock1.a))/Stock1.a)*Stock1.y0+Stock1.a*V1+np.matmul(E,J);
    Kvar = ((1-np.exp(-Stock1.a))/Stock1.a)*Stock1.y0+Stock1.b-Stock1.b*((1-np.exp(-Stock1.a))/Stock1.a)+j1;
    print(Stock1.b)


    return(Kvar*10000)

In [157]:
def Continuous3():
   ####Continuous solution without regime switching
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    Stock1 = Stock(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    Stock2 = Stock1
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    Jump1 = Jump_Merton(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82,0.05,0.086,0.2)
    Jump2 = Jump1
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])
    Regime = Regime2(Q);T=1;


    n = 101 # time step of integration
    X = np.linspace(0,1,n+1)
    Phi_Matrix=[]
    E = np.ones(Q.ndim)                     
    for i in range(len(X)):
        v1 = Stock1.mean(X[i]);v2 = Stock2.mean(X[i])
        phi = np.diag(np.array([v1,v2]))
        pphi = np.matmul(linalg.expm(Q.transpose()*(X[i])),phi)
        Phi = np.matmul(E,pphi)
        Phi_Matrix.append(Phi)
        #print(Phi)
        
    
    a = list(zip(*Phi_Matrix))
    #print(a)
    V1 = np.trapz(a,dx=0.01);
    j1 = Jump1.mean_com(); j2 = Jump2.mean_com();
    V2 = np.diag(np.array([j1,j2]));
    J = np.matmul(linalg.expm(Q.transpose()*T),V2);
    Kvar = ((1-np.exp(-Stock1.a))/Stock1.a)*Stock1.y0+Stock1.b-Stock1.b*((1-np.exp(-Stock1.a))/Stock1.a)+j1;
    print(Stock1.b)


    return(Kvar*10000)

In [158]:
def regime_VS2(AF):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
   
    Delta = 1/AF
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])#transition matrix
    #Stock1 = Stock(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    #Stock2 = Stock(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82)
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    #Jump1 = Jump_Merton(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82,0.05,0.086,0.)
    #Jump2 = Jump_Merton(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82,0.06,0.086,0.3)
    Regime = Regime2(Q);
    ################################################################################
    
    U = np.array([0,0])#initialize
    T = 1
    for k in range(0,AF*T):
        t_k = (k+1)*Delta
        Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
        Stock2 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82)
        Jump1 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82,0.05,0.086,0.3)
        Jump2 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82,0.05,0.086,0.3)
        R = np.diag([np.exp(Stock1.r*Delta),np.exp(Stock2.r*Delta)])# matrix of interest rate
        Phi1_2j,Phi1_1j,Phi1_0j = Phi1(t_k,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2) 
        Phi2_,H1 = Phi2(t_k,AF,Q,Delta,Stock1,Stock2,Regime)
        if t_k == Delta:
            M = Stock1.M(-2j,0)
            uk = Phi1_2j[1]*np.exp(M*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
        else:
            uk = np.multiply(Phi1_2j[1],Phi2_[1])*np.exp(H1*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
            
        U = U+Uk
        
        
        
       
    K = (U/T)*10000
    return K

In [159]:
def regime_VS3(AF):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
   
    Delta = 1/AF
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])#transition matrix
    #Stock1 = Stock(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    #Stock2 = Stock(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82)
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    #Jump1 = Jump_Merton(100,0.087**2,252,0.06,0.14,3.46,0.006704,1,-0.82,0.05,0.086,0.)
    #Jump2 = Jump_Merton(100,0.087**2,252,0.03,0.14,3.46,0.002852,1,-0.82,0.06,0.086,0.3)
    Regime = Regime2(Q);
    ################################################################################
    
    U = np.array([0,0])#initialize
    T = 1
    for k in range(0,AF*T):
        t_k = (k+1)*Delta
        Stock1 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
        Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82)
        Jump1 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82,0.05,0.086,0.)
        Jump2 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82,0.05,0.086,0.)
        R = np.diag([np.exp(Stock1.r*Delta),np.exp(Stock2.r*Delta)])# matrix of interest rate
        Phi1_2j,Phi1_1j,Phi1_0j = Phi1(t_k,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2) 
        Phi2_,H1 = Phi2(t_k,AF,Q,Delta,Stock1,Stock2,Regime)
        if t_k == Delta:
            M = Stock1.M(-2j,0)
            uk = Phi1_2j[1]*np.exp(M*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
        else:
            uk = np.multiply(Phi1_2j[1],Phi2_[1])*np.exp(H1*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
            #Uk = np.matmul(R,uk)
            Uk = uk
            
        U = U+Uk
        
    K = (U/T)*10000
    return K
        

In [160]:
def discreet2(AF):
    
    
    Kvar = []
    
    for t in AF:
        K = regime_VS2(t)
        Kvar.append(K)
     
    return(Kvar)

In [161]:
def discreet3(AF):
    
    
    Kvar = []
    
    for t in AF:
        K = regime_VS3(t)
        Kvar.append(K)
     
    return(Kvar)

In [166]:
# final main()
from VS_class2 import Stock, Regime2, Jump_Merton, Jump_Kou
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import linalg

AF = np.array([4,12,26,52,252])

# calculate discrete sols based AF
Kvar_d = discreet(AF)
K_d = list(zip(*Kvar_d))
# calculate discrete sols based AF, without regime switching
Kvar_d1 = discreet2(AF)
K_d1 = list(zip(*Kvar_d1))
Kvar_d2 = discreet3(AF)
K_d2 = list(zip(*Kvar_d2))

print("Here we display the result:")
# print("The sequence of the observing frequency is：")
# print(AF)
# print("Kvar with considertion of regime switching, state one:")
# print(K_d[0])
# print("Kvar with considertion of regime switching, state two:")
# print(K_d[1])
# print("Kvar without considertion of regime switching, state one:")
# print(K_d1[0])
# print("Kvar without considertion of regime switching, state two:")
# print(K_d2[0])
# # calculate continuous sols and copy to len(AF)
# # continuous solution of regime situation
print("Kvar with considertion of regime switching, continuous:")
K = Continuous()
print(K)
# continuous solution of no-regime situation
print("Kvar without considertion of regime switching, continuous:")
K2 = Continuous2()
print(K2)
print("Kvar without considertion of regime switching, continuous:")
K3 = Continuous3()
print(K3)

Here we display the result:
Kvar with considertion of regime switching, continuous:
[[1.82344910e-03 7.78754518e-05]
 [6.23003615e-04 6.78098193e-04]]
[0.00343605 0.00174557] [0.00955731 0.00515567]
[95.57305552 51.55672834]
Kvar without considertion of regime switching, continuous:
0.002852
51.62046766635653
Kvar without considertion of regime switching, continuous:
0.006704
89.25342559495408
